## Message Translator (PayloadTransformer)

This notebook demonstrates the Message Translator pattern using a `PayloadTransformer` with `JxScript`, `JObj`, and `jx.assign` for structured JSONata expression building. This approach provides better type safety, readability, and maintainability compared to raw JSONata strings.

**Input Format**: `CustomerRequest` - A customer service request  
**Output Format**: `InternalTicket` - An internal support ticket format

**Transformation Benefits with jx.assign:**
- **Readability**: Complex expressions are broken into named variables
- **Reusability**: Intermediate results can be reused throughout the transformation
- **Debugging**: Easier to understand and debug transformation logic
- **Maintainability**: Changes to specific parts of the transformation are isolated

The flow will be:

[ProbeAgent] --`CustomerRequest`-> (default_topic) -> [Translator Agent/TranslatorAgent:wire_message] --`InternalTicket`-> (internal_tickets)

In [ ]:
from pydantic import BaseModel
from typing import Optional
from datetime import datetime

# Input message format - what customers send
class CustomerRequest(BaseModel):
    customer_name: str
    email: str
    subject: str
    description: str
    priority: str  # "low", "medium", "high"

# Output message format - internal ticket system
class InternalTicket(BaseModel):
    ticket_id: str
    customer_info: dict
    title: str
    details: str
    severity: int  # 1=high, 2=medium, 3=low
    created_at: str
    status: str = "open"

In [ ]:
from rustic_ai.core.agents.eip.basic_wiring_agent import BasicWiringAgent
from rustic_ai.core.guild.builders import AgentBuilder, GuildBuilder, RouteBuilder
from rustic_ai.core.guild.dsl import GuildTopics
from rustic_ai.core.utils.basic_class_utils import get_qualified_class_name
from rustic_ai.core.utils import jx
from rustic_ai.core.utils.jexpr import JObj, JxScript

# Create the translator agent
translator_agent = (
    AgentBuilder(BasicWiringAgent)
    .set_id("TranslatorAgent")
    .set_name("Message Translator")
    .set_description("Translates customer requests into internal ticket format using PayloadTransformer")
    .build_spec()
)


In [ ]:

# Create a JxScript with JObj that transforms CustomerRequest to InternalTicket
# Using jx.assign to break down complex transformation into readable variables
payload_transformation = JxScript(
    # Assign intermediate variables for better readability
    jx.assign("$timestamp", jx.millis()),
    jx.assign("$random_value", jx.JExpr("$random()")),
    jx.assign("$random_suffix", jx.substring(jx.string("$random_value"), 2, 6)),
    jx.assign("$timestamp_str", jx.string("$timestamp")),
    jx.assign("$ticket_prefix", jx.substringBefore("$timestamp_str", ".")),
    jx.assign("$generated_ticket_id", jx.JExpr('"TKT-" & $ticket_prefix & "-" & $random_suffix')),
    
    # Assign priority-to-severity mapping using nested ternary
    jx.assign("$severity_level", jx.ternary(
        jx.JExpr("priority") == "high", 
        1, 
        jx.ternary(jx.JExpr("priority") == "medium", 2, 3)
    )),
    
    # Assign customer information object
    jx.assign("$customer_details", JObj({
        "name": jx.JExpr("customer_name"),
        "email": jx.JExpr("email")
    })),
    
    # Final transformation object using assigned variables
    JObj({
        "ticket_id": jx.JExpr("$generated_ticket_id"),
        "customer_info": jx.JExpr("$customer_details"),
        "title": jx.JExpr("subject"),
        "details": jx.JExpr("description"),
        "severity": jx.JExpr("$severity_level"),
        "created_at": jx.now(),
        "status": jx.JExpr('"open"')
    })
)


In [ ]:

# Create the routing rule with PayloadTransformer
translation_rule = (
    RouteBuilder(translator_agent)
    .filter_on_origin(origin_message_format=get_qualified_class_name(CustomerRequest))
    .set_payload_transformer(InternalTicket, payload_transformation)
    .set_destination_topics("internal_tickets")
    .build()
)


In [ ]:

# Create the guild
guild = (
    GuildBuilder(
        guild_id="MessageTranslatorPayloadGuild",
        guild_name="Message Translator (Payload) Guild",
        guild_description="A guild demonstrating message translation using PayloadTransformer.",
    )
    .add_agent_spec(translator_agent)
    .add_route(translation_rule)
    .launch("myorg")
)

In [ ]:
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent

guild_default_topic = GuildTopics.DEFAULT_TOPICS[0]

probe_spec = (
    AgentBuilder(ProbeAgent)
    .set_id("ProbeAgent")
    .set_name("Probe Agent")
    .set_description("A probe agent to test message translation.")
    .add_additional_topic("internal_tickets")
    .build_spec()
)

probe_agent: ProbeAgent = guild._add_local_agent(probe_spec)  # type: ignore

In [ ]:
# Send a customer request
customer_request = CustomerRequest(
    customer_name="John Doe",
    email="john.doe@example.com",
    subject="Login Issue",
    description="I can't log into my account. Getting 'invalid credentials' error.",
    priority="high"
)

probe_agent.publish_with_guild_route(
    payload=customer_request,
    topic=guild_default_topic,
)

In [ ]:
# Check the translated message
probe_agent.print_message_history()

In [ ]:
last_message = probe_agent.get_messages()[-1]
print(f"Format: {last_message.format}")
print(f"Payload: {last_message.payload}")

In [ ]:
# Send another customer request with different priority
customer_request_2 = CustomerRequest(
    customer_name="Jane Smith",
    email="jane.smith@company.com",
    subject="Feature Request",
    description="Would like to request a new dashboard feature for analytics.",
    priority="low"
)

probe_agent.publish_with_guild_route(
    payload=customer_request_2,
    topic=guild_default_topic,
)

In [ ]:
# Check all messages to see both translations
probe_agent.print_message_history()

In [ ]:
last_message = probe_agent.get_messages()[-1]
print(f"Format: {last_message.format}")
print(f"Payload: {last_message.payload}")